# Load data into a notebook from different sources (Python)

Before you can start analyzing your data, you have to load the data from a data source. You can store your data in many different data sources. This reference notebook shows you how to load and integrate data in a notebook from the following data sources:
-  Object Storage V3
-  dashDB
-  Cloudant
-  MongoDB
-  PostgreSQL

The notebook sample code shows you how to load data into a notebook by using Python and the PySpark stack. You can copy and paste these code snippets into the notebook you are developing.

## Table of contents

- [Getpass](#getpass)
- [Load data from Object Storage V3](#osv3)
  - [Load data by using Python](#osv3_python)
  - [Load data by using PySpark](#osv3_pyspark)
  - [Load data by using Stocator](#osv3_stocator)
- [Load data from dashDB](#dashdb)
  - [Load data by using PySpark](#dashdb_pyspark)
- [Load data from a Cloudant database](#cloudant)
  - [Load data by using Python](#cloudant_python)
  - [Load data by using PySpark](#cloudant_pyspark)
- [Load data from MongoDB](#mongodb)
- [Load data from a PostgreSQL database](#postgresql)
- [Summary](#summary)

<a id="getpass"></a>
## Getpass
The `getpass()` function can be used to enter a password into your credentials. This is useful for sharing notebooks or presenting when you do not want to publicly display your password. First, we import `getpass`, then we replace the value of our `credentials` dictionary's `password` key with `getpass.getpass()`. Now instead of the cell displaying your password, you will be prompted to enter it upon execution.

In [ ]:
import getpass

#insert this into password's value in dictionary
password = getpass.getpass()
#display that it works
print password

<a id="osv3"></a>
## Load data from Object Storage V3
IBM® Object Storage for Bluemix® provides you with access to a fully provisioned Swift Object Storage account to manage your data. Object Storage uses OpenStack Identity (Keystone) for authentication and can be accessed directly by using [OpenStack Object Storage (Swift) API v3](http://developer.openstack.org/api-ref-identity-v3.html#credentials-v3). 

When you load data for use in a notebook, the data file is stored in the Object Storage instance associated with your Spark service.

Click the next code cell to set the focus on the cell. Now add the credentials to access the data file to this code cell by selecting **Palette>Data Sources** and clicking the `Insert to code` function below the data file in the **Data Source** pane.

When you select the `Insert to code` function, a code cell with a Python dictionary is created for you. Adjust the credentials in the dictionary to correspond with the credentials inserted by the `Insert to code function` and run the dictionary code cell. The access credentials to the Object Storage instance in the dictionary are provided for later usage.

<a id="osv3_python"></a>
### Load data by using Python

Run the next cells to load the data from a file in Object Storage by using Python functions: 

In [ ]:
import requests, StringIO, json

def get_file_content(credentials):
    """For given credentials, this functions returns a StringIO object containg the file content 
    from the associated Bluemix Object Storage V3."""

    url1 = ''.join([credentials['auth_url'], '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
                                  'password': {'user': {'name': credentials['username'],
                                                        'domain': {'id': credentials['domain_id']},
                                                        'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                if(e2['interface']=='public'and e2['region']==credentials['region']):
                    url2 = ''.join([e2['url'],'/', credentials['container'], '/', credentials['filename']])
                    print url2

    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO.StringIO(resp2.content)

In [ ]:
import pandas as pd

data_df = pd.read_csv(get_file_content(credentials))
data_df.head()

Now your data is in a `pandas.DataFrame` and you can begin analyzing it. 

<a id="osv3_pyspark"></a>
### Load data by using PySpark

Before you can access data in the data file in Object Storage by using the [`SparkContext`](https://spark.apache.org/docs/1.6.0/api/python/pyspark.html#pyspark.SparkContext) object, you must set the Hadoop configuration by using the following configuration function:

In [ ]:
def set_hadoop_config(credentials):
    """This function sets the Hadoop configuration with given credentials, 
    so it is possible to access data using SparkContext"""
    
    prefix = "fs.swift.service." + credentials['name']
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + ".auth.url", credentials['auth_url']+'/v3/auth/tokens')
    hconf.set(prefix + ".auth.endpoint.prefix", "endpoints")
    hconf.set(prefix + ".tenant", credentials['project_id'])
    hconf.set(prefix + ".username", credentials['user_id'])
    hconf.set(prefix + ".password", credentials['password'])
    hconf.setInt(prefix + ".http.port", 8080)
    hconf.set(prefix + ".region", credentials['region'])
    hconf.setBoolean(prefix + ".public", True)

Set the Hadoop configuration and give it a name, for example, `keystone`:

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# you can choose any alphanumeric name
credentials['name'] = 'keystone'
set_hadoop_config(credentials)

data_rdd = sc.textFile("swift://" + credentials['container'] + "." + credentials['name'] + "/" + credentials['filename'])
data_rdd.take(5)

Now your data is in a `pyspark.RDD` and you can begin analyzing it. 

<a id="osv3_stocator"></a>
### Load data using Stocator
Stocator is a storage connector for Spark that eliminates some of the unnecessary Hadoop drivers that are not needed to interact with object storage. Stocator's Hadoop configuration can be set by using the following configuration function:

In [ ]:
def set_hadoop_config(name, credentials):
    prefix = "fs.swift2d.service." + name 
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set("fs.swift2d.impl", "com.ibm.stocator.fs.ObjectStoreFileSystem")
    hconf.set(prefix + ".auth.url", credentials['auth_url']+'/v3/auth/tokens')
    hconf.set(prefix + ".auth.endpoint.prefix", "endpoints")
    hconf.set(prefix + ".tenant", credentials['project_id'])
    hconf.set(prefix + ".username", credentials['user_id'])
    hconf.set(prefix + ".password", credentials['password'])
    hconf.setInt(prefix + ".http.port", 8080)
    hconf.set(prefix + ".region", credentials['region'])
    hconf.setBoolean(prefix + ".public", True)

Set the Hadoop configuration and check to see that it is set to stocator. The output of this cell should read `com.ibm.stocator.fs.ObjectStoreFileSystem`.

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

set_hadoop_config("sparksql", credentials)
print sc._jsc.hadoopConfiguration().get("fs.swift2d.impl")

Run this cell to load the data.

In [ ]:
FILENAME = credentials["filename"]
FILENAME2D = "swift2d://notebooks.sparksql/" + FILENAME

data_rdd = sc.textFile(FILENAME2D)
data_rdd.take(5)

Now your data is in a `pyspark.RDD` and you can begin analyzing it. 

<a id="dashdb"></a>
## Load data from dashDB

dashDB is a data warehousing and analytics solution. Use dashDB to store relational data, including special data types such as geospatial data. You can leverage the in-memory database technology to use both columnar and row-based tables. 

You must have an IBM dashdDB for Bluemix service instance. In the notebook, select **Palette>Data Sources**. Click **Add Source**, select **From Bluemix**, and choose your dashDB instance. The dashDB instance name appears in the **Data Source** pane. 

Click the next code cell and use the `Insert to code` function below the dashDB instance name in the **Data Source** pane to add the dashDB credentials. 

When you select the `Insert to code` function, a code cell with a Python dictionary is created for you. Adjust the credentials in the dictionary to correspond with the credentials inserted by the `Insert to code function` and run the dictionary code cell. The access credentials to the dashDB  instance in the dictionary are provided for later usage.

<a id="dashdb_pyspark"></a>
### Load data by using PySpark

Add the credentials of your dashDB instance that contains your data and run the next cell to load this data. Be sure to set the `TABLENAME` variable to the name of the table in your DashDB you would like to access.

The code in the cell reads the credentials and loads the data from dashBD into a DataFrame data structure.

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

TABLENAME = "<name>"

props = {}
props['user'] = credentials_2['username']
props['password'] = credentials_2['password']

table = credentials_2['username'] + "." + TABLENAME

data_df = sqlContext.read.jdbc(credentials_2['jdbcurl'],table,properties=props)
data_df.printSchema()

In [ ]:
data_df.take(5)

Now your data is in a `pyspark.sql.DataFrame` and you can analyze it. 

<a id="cloudant"></a>
## Load data from a Cloudant database
Cloudant is a NoSQL database as a service (DBaaS) built to scale globally, run nonstop, and handle a wide variety of data types like JSON, full-text, and geospatial. Cloudant NoSQL DB is an operational data store optimized to handle concurrent reads and  writes and to provide high availability and data durability.

You must have an IBM Cloudant NoSQL Database for Bluemix service instance. In the notebook, select **Palette>Data Sources**. Click **Add Source**, select **From Bluemix**, and choose your Cloudant NoSQL DB instance. The Cloudant NoSQL DB instance name appears in the **Data Source** pane. 

Click the next code cell and use the `Insert to code` function below the Cloudant NoSQL DB instance name in the **Data Source** pane to add the Cloudant NoSQL DB instance credentials. 

Adjust the credentials in the Python dictionary, which is prepared for you, to correspond with the credentials inserted by the `Insert to code` function and run the dictionary code cell. The access credentials to your Cloudant NoSQL DB instance in the dictionary are provided for convenience for later usage.

<a id="cloudant_python"></a>
### Load data by using Python

Before you begin loading data from a Cloudant NoSQL DB instance to your notebook, ensure that you are using the latest database version. Do not use Cloudant 0.5.10 or earlier. For more information see [the Python library for Cloudant and CouchDB](https://github.com/cloudant/python-cloudant).

Install the `cloudant` package:

In [ ]:
!pip install --user cloudant

In [ ]:
from cloudant.client import Cloudant
from cloudant.result import Result
import pandas as pd, json

client = Cloudant(credentials_3['username'], credentials_3['password'], url=credentials_3['url'])
client.connect()

List all existing databases:

In [ ]:
client.all_dbs()

In [ ]:
# fill in database name 
db_name = 'test_db'
my_database = client[db_name]
result_collection = Result(my_database.all_docs, include_docs=True)
data_df = pd.DataFrame([item['doc'] for item in result_collection])
data_df.head()

Now your data is in a `pandas.DataFrame` and you can start analyzing it.

<a id="cloudant_pyspark"></a>
### Load data by using PySpark

Add the credentials of your Cloudant NoSQL DB instance that contains your data and run the next cell to load this data.

The code in the cell reads the credentials and loads the data from Cloudant into a DataFrame data structure.

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# fill in database name 
db_name = "test_db"
data_df = sqlContext.read.format("com.cloudant.spark")\
.option("cloudant.host",credentials_3['host'])\
.option("cloudant.username",credentials_3['username'])\
.option("cloudant.password",credentials_3['password'])\
.load(db_name)

data_df.printSchema()

In [ ]:
data_df.take(5)

Now your data in a `pyspark.sql.DataFrame` and you can start analyzing it.

<a id="mongodb"></a>
## Load data from MongoDB
MongoDB is an NoSQL, document-oriented database. MongoDB favors JSON-like documents over the traditional relational table database structure.

MongoDB is offered as a Bluemix service which must be paired with an existing [Compose](https://www.compose.com/) account.

First, we will install and import the necessary drivers and packages.

In [ ]:
!pip install --user pymongo

import ssl
import json
import pymongo
from pymongo import MongoClient

The following connection credentials can be found on your Compose account.

In [ ]:
host = '<host>'
database = '<db>'
collection = '<col>'

In [ ]:
client = MongoClient(host+database, ssl=True, ssl_cert_reqs=ssl.CERT_NONE)
db = client[database]
col = db[collection]

The following cells read the credentials and load the data from the specified collection into a Spark DataFrame.

In [ ]:
cursor = col.find()
json_content = []
json_file = 'mongo.json'

for doc in cursor:
    doc['_id'] = str(doc['_id'])
    json_content.append(doc)
data = json.dumps(json_content)

with open(json_file, 'w') as text_file:
      text_file.write(data)

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

df = sqlContext.read.json(json_file)
df.printSchema()
df.show(5)

Once the data is in the DataFrame, we can clean up the json file we created.

In [ ]:
# !rm <json_file>

Now your data is in a `pyspark.sql.DataFrame` and you can start analyzing it.

<a id="postgresql"></a>
## Load data from a PostgreSQL database
PostgreSQL is an object-relational database system offered as a Bluemix service. It must be paired with an existing [Compose](https://www.compose.com/) account to be used.

The following jar must be added in order for the PostgreSQL jdbc driver to work. However, Python notebooks do not support the `%Addjar` magic function. To load the jar, run the following cell in a Scala notebook running on the same Spark instance.

In [ ]:
#%Addjar https://jdbc.postgresql.org/download/postgresql-9.4.1208.jre6.jar 

After that, we will set the credentials of the table we would like to access. These can be found in your Compose account.

In [ ]:
host = '<host>'
port = '<port>'
user = '<user>'
password = '<password>'
dbname = '<db>'
dbtable = '<table>'

Now we can run this cell to have the jdbc driver load your data.

In [ ]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

df = sqlContext.read.format('jdbc')\
                    .options(url='jdbc:postgresql://'+host+':'+port+'/'+dbname+'?user='+user+'&password='+password, dbtable=dbtable)\
                    .load()
df.printSchema()
df.show(5)

Now your data is in a `pyspark.sql.DataFrame` and you can start analyzing it.

<a id="summary"></a>
## Summary

In this notebook, you learned how to load data from an Object Storage V3, dashDB, Cloudant, MongoDB or PostgreSQL instance to a notebook.